# Importing the necessary library to process the Audio files and extract the features

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from glob import glob
import librosa as lr
import re
from IPython.display import Audio
from scipy.fftpack import fft, rfft
from scipy import stats
from librosa.core import piptrack
from librosa.feature import mfcc
from scipy.stats.mstats import gmean

# Appending the directories for the audio files in a list, to open them later.

In [2]:
audio_files = glob('..//audios_to_test//friends_voices//*.wav')

In [3]:
len(audio_files)

183

In [4]:
audio_files[1]

'..//audios_to_test//friends_voices\\f (10).wav'

## Opening the audio files using librosa, the autputs are:
- audio = audio time series
- sampling_rate = sampling rate of audio

In [5]:
# TO open an audio file at a time and listen to it in the notebook
audio, sampling_rate = lr.load(audio_files[18])
Audio(audio, rate=sampling_rate)
print(audio_files[15])

..//audios_to_test//friends_voices\f1 (5).wav


In [6]:
# list_of_dict2 = []
# for file in range(0, len(audio_files), 1):
#     audio, sampling_rate = lr.load(audio_files[file])
#     pitches, magnitudes = piptrack(y=audio, sr=sampling_rate)
#     mfccs = mfcc(y=audio, sr=sampling_rate)
#     rms = lr.feature.rms(audio)
#     new_dict = {}
#     new_dict[3] = magnitudes.mean() 
#     new_dict[5] = mfccs.max()
#     new_dict[6] = rms.mean()
#     new_dict[7] = rms.min()
#     new_dict[9] = rms.std()
#     if audio_files[file].startswith('..//audios_to_test//friends_voices\\f'):
#         new_dict['Gender'] = 1
#     elif  audio_files[file].startswith('..//audios_to_test//friends_voices\\m'):
#         new_dict['Gender'] = 0
#     list_of_dict2.append(new_dict)

In [7]:
# check = pd.DataFrame(list_of_dict2)
# check.corr()

## To plot each audio file in the time domain

In [8]:
for file in range(0, len(audio_files), 1):
    audio, sampling_rate = lr.load(audio_files[file])
    time = np.arange(0, len(audio)) / sampling_rate
    #print(audio_files[file])
    #plt.plot(time, audio)
    #plt.xlabel('Time (secs)')
    #plt.show()

# Creating a function to extract the features needed out of an Audio file and then append the features into a dictionary

In [9]:
def feature_extractor(audio: np.ndarray, sampling_rate: int) -> dict:
    frequency_spectrum = np.abs(np.fft.rfft(audio))
    frequency = np.fft.rfftfreq(len(audio), d=1 / sampling_rate)
    frequency_spectrum = np.abs(frequency_spectrum)
    amplitude = frequency_spectrum / frequency_spectrum.sum()
    mean_frequency = (frequency * amplitude).sum()
    freq_standerd_deviation = np.sqrt(np.sum(amplitude * ((frequency - mean_frequency) ** 2)))
    amplitude_cumulative_sum = np.cumsum(amplitude)
    median_frequency = frequency[len(amplitude_cumulative_sum[amplitude_cumulative_sum <= 0.5]) + 1]
    mode_frequency = frequency[amplitude.argmax()]
    quartile_25 = frequency[len(amplitude_cumulative_sum[amplitude_cumulative_sum <= 0.25]) + 1]
    quartile_75 = frequency[len(amplitude_cumulative_sum[amplitude_cumulative_sum <= 0.75]) + 1]
    interquartile_range = quartile_75 - quartile_25
    deviation_from_mean = amplitude - amplitude.mean()
    amplitude_std = amplitude.std()
    skewness = ((deviation_from_mean ** 3).sum() / (len(frequency_spectrum) - 1)) / amplitude_std ** 3
    kurtosis = ((deviation_from_mean ** 4).sum() / (len(frequency_spectrum) - 1)) / amplitude_std ** 4
    centroid_frequency = lr.feature.spectral_centroid(y=audio, sr=sampling_rate)
    spectral_flatness = lr.feature.spectral_flatness(y=audio)
    pitches, magnitudes = piptrack(y=audio, sr=sampling_rate, fmax = 280)
    mfccs = mfcc(y=audio, sr=sampling_rate)
    root_mean_square = lr.feature.rms(audio)

    dictionary_of_features = {
        'Mean_freq': mean_frequency/1000,
        'Std': freq_standerd_deviation/1000,
        'Median_freq': median_frequency/1000,
        'Mode_freq': mode_frequency/1000,
        'First_quartile': quartile_25/1000,
        'Third_quartile': quartile_75/1000,
        'Interquantile_range': interquartile_range/1000,
        'Skewness': skewness,
        'Kurtosis': kurtosis,
        'Centroid_freq': np.mean(centroid_frequency)/1000,
        'Spectral_flatness_measure': np.mean(spectral_flatness),
        #'Mean_fundamental_freq': (pitches[np.nonzero(pitches)].mean())/1000,
        #'Min_fundamental_freq': (pitches[np.nonzero(pitches)].min())/1000,
        #'Max_fundamental_freq': (pitches[np.nonzero(pitches)].max())/1000,
        #'Mean_MFCCs' : mfccs.mean(),                            #Mel-frequency cepstral coefficients (MFCCs)
        'Std_MFCCs': mfccs.std(),
        'Root_mean_square': root_mean_square.mean(),
        'Mean_Magnitude' : magnitudes.mean(),
        'Max_MFCCs' : mfccs.max(),
        'Mean_RMS': root_mean_square.mean(),
        'Min_RMS': root_mean_square.min(),
        'Std_RMS': root_mean_square.std(),
    }

    return dictionary_of_features

# To open all the audio files, plot the frequency domain of ech file and apply the feature_extractor function to pull the features from the audio files and then append them into a list to use in creating a dataframe later.

### I added a condition to the loop to use the file name and append a new key: values pair in the dictionary, if the file starts with 'f' it appends 1 to a Gender key in the dictionary, if the file starts with anything else 'm', it appends 0 to the Gender key in the dictionary. 

In [10]:
list_of_dict = []
for file in range(0, len(audio_files), 1):
    audio, sampling_rate = lr.load(audio_files[file])
    #frequencies = rfft(audio)
    #plt.plot(abs(frequencies))
    #plt.show()
    #print(audio_files[file])
    dictionary_of_features = feature_extractor(audio, sampling_rate)
    if audio_files[file].startswith('..//audios_to_test//friends_voices\\f'):
        dictionary_of_features['Gender'] = 1
    elif  audio_files[file].startswith('..//audios_to_test//friends_voices\\m'):
        dictionary_of_features['Gender'] = 0
    list_of_dict.append(dictionary_of_features)

## Printing the list of dictionaries to make sure that all the dictionaries are in it.

In [11]:
#list_of_dict

## Creating a dataframe for the features extracted from the audio files

In [12]:
friends_voices = pd.DataFrame(list_of_dict)

In [13]:
friends_voices

,Centroid_freq,First_quartile,Gender,Interquantile_range,Kurtosis,Max_MFCCs,Mean_Magnitude,Mean_RMS,Mean_freq,Median_freq,Min_RMS,Mode_freq,Root_mean_square,Skewness,Spectral_flatness_measure,Std,Std_MFCCs,Std_RMS,Third_quartile
0,1.080912,0.073333,1,1.057900,169.249380,210.773499,0.002515,0.016834,0.799334,0.389141,0.001797,0.019619,0.016834,11.132721,2.522256e-05,0.996447,99.014488,0.019451,1.131233
1,1.075254,0.112250,1,1.243750,109.336139,239.328064,0.002583,0.016529,0.926404,0.574500,0.001869,0.017250,0.016529,8.663482,7.507778e-06,1.003345,104.859772,0.016728,1.356000
2,1.188634,0.123448,1,1.968132,223.629151,224.032501,0.002373,0.015104,1.118611,0.487723,0.001801,0.016552,0.015104,11.788378,4.365468e-05,1.192840,103.268341,0.012596,2.091580
3,1.039470,0.098399,1,1.167726,118.146152,219.185089,0.002066,0.012939,0.900871,0.487464,0.001897,0.048000,0.012939,9.089269,4.571596e-05,1.033259,107.022255,0.011266,1.266126
4,1.084815,0.179667,1,1.333667,104.043347,231.746552,0.002823,0.014253,1.021227,0.662333,0.001890,0.017667,0.014253,8.128698,1.851003e-05,1.046034,103.368408,0.011550,1.513333
5,1.152530,0.205894,1,1.635782,108.153429,250.349243,0.001962,0.011980,1.198078,0.777680,0.001771,0.022737,0.011980,8.300788,1.274553e-05,1.143682,109.586700,0.008400,1.841675
6,1.123593,0.167556,1,1.405111,179.949207,228.735748,0.002134,0.011687,1.025563,0.526222,0.001862,0.019333,0.011687,10.758468,2.908826e-05,1.104131,105.307213,0.010220,1.572667
7,1.097525,0.188255,1,1.137700,100.670705,222.651840,0.002296,0.011481,0.946246,0.529531,0.001869,0.019404,0.011481,7.995139,5.092275e-05,1.012016,106.539879,0.009928,1.325955
8,1.137168,0.148571,1,1.623501,156.814999,225.002167,0.001660,0.011122,1.078645,0.534854,0.001948,0.013877,0.011122,10.033042,2.079796e-05,1.165513,108.632835,0.009243,1.772072
9,1.154020,0.215651,1,1.433560,97.886893,233.622650,0.002199,0.010861,1.072147,0.618258,0.001761,0.018087,0.010861,7.583603,1.042173e-05,1.067420,109.815269,0.008657,1.649211


In [14]:
friends_voices.to_csv('..//datasets//friends_voices.csv', index=False)